In [130]:
import numpy as np
import os
import cv2
import random

from os.path import join as pjoin
from functools import partial
from tqdm import tqdm
import matplotlib.pyplot as plt

In [131]:
def generator(path,
              size=256,
              crop_size=32,
              is_test=False):
    image_paths = [pjoin(path, img_path) for img_path in os.listdir(path)]
    img_path = random.choice(image_paths)
    img = cv2.imread(img_path, 0) #read BGR
    img = cv2.resize(img, (size, size))
    while 1:
        i, j = random.choice(list(range(size//crop_size))), random.choice(list(range(size//crop_size)))
        x = img[i*crop_size: (i+1)*crop_size, j*crop_size: (j+1)*crop_size].flatten()
        assert(x.shape[0] == (crop_size)**2)
        x = 2*x/255 - 1
        yield x[np.newaxis, ...]
            

In [197]:
dataset_path = "VOCdevkit/VOC2012/JPEGImages/"
gen = generator(dataset_path)
dataset_size = len(os.listdir(dataset_path))
print(next(gen))

[[-0.98431373 -0.14509804 -0.24705882 ... -0.83529412 -0.86666667
  -0.83529412]]


In [198]:
def glorot_uniform(input_layers, output_layers):
    limit = np.sqrt(6 / (input_layers + output_layers))
    return partial(np.random.uniform, low=-limit, high=limit)

class Autoencoder():
    def __init__(self, 
                 input_layers=1024, 
                 mid_layers=256,
                 lr=5e-5,
                 max_err=1000.0,
                 phase='train'):
        self.input_layers = input_layers
        self.mid_layers = mid_layers
        self.initializer = glorot_uniform(input_layers, mid_layers)
        self.phase=phase
        self.lr = lr
        self.max_err = max_err
        self.loss = lambda x, y: ((x - y) ** 2)
        self.build()
    
    def build(self):
        self.W1 = self.initializer(size=[self.input_layers, self.mid_layers])
        self.W2 = self.initializer(size=[self.mid_layers, self.input_layers])
    
    def __call__(self, inp):
        mid, res = self.forward(inp)
        err = self.loss(inp, res)
            
        if self.phase == 'train':
            if abs(err.mean()) < self.max_err:
                self.backward(inp, mid, np.abs(res-inp))
                return err.mean()
            else:
                return self.max_err
        else:
            return res
    
    def forward(self, inp):
        mid = self.encode(inp)
        return mid, self.decode(mid)
    
    def backward(self, inp, mid, err):
        lr = 1/np.dot(mid, mid.T)**2#lr = self.lr
        self.W2 -= lr * np.dot(mid.T, err)
        lr = 1/np.dot(inp, inp.T)**2#lr = self.lr
        self.W1 -= lr * np.dot(np.dot(inp.T, err), self.W2.T)
        self.W2 /= np.linalg.norm(self.W2)
        self.W1 /= np.linalg.norm(self.W1)
                          
    def encode(self, inp):
        return np.dot(inp, self.W1)
    
    def decode(self, mid):
        return np.dot(mid, self.W2)

In [ ]:
model = Autoencoder()
momentum = 0.95
errors = []
moving_err = None
it_count = 10000
for it in range(it_count):
    inp = next(gen)
    err = model(inp)
    moving_err = momentum * moving_err + err * (1. - momentum) if moving_err else err
    errors.append(moving_err)
    print("EPOCH {} MSE {}".format(it, moving_err))

x = np.arange(it_count)
plt.plot(x, np.array(errors))
idx = np.argmin(errors)
print("BEST ERROR {}".format(errors[idx]))
plt.plot(x[idx], errors[idx], 'rx--', linewidth=2, markersize=12)
plt.show()

EPOCH 0 MSE 0.5192832619679595
EPOCH 1 MSE 0.5283087561133519
EPOCH 2 MSE 0.5193903023133296
EPOCH 3 MSE 0.5178447641636251
EPOCH 4 MSE 0.5216224321677161
EPOCH 5 MSE 0.5262032471430366
EPOCH 6 MSE 0.5279233295832184
EPOCH 7 MSE 0.5148353910289637
EPOCH 8 MSE 0.5099941775682248
EPOCH 9 MSE 0.4949052526318499
EPOCH 10 MSE 0.49035692296075695
EPOCH 11 MSE 0.47432012068944507
EPOCH 12 MSE 0.48067338684865235
EPOCH 13 MSE 0.4838131181719973
EPOCH 14 MSE 0.4783870727712739
EPOCH 15 MSE 0.4743749402883457
EPOCH 16 MSE 0.46876063338292434
EPOCH 17 MSE 0.45336688993303226
EPOCH 18 MSE 0.4445804684142165
EPOCH 19 MSE 0.4501712771136649
EPOCH 20 MSE 0.44349472190302436
EPOCH 21 MSE 0.4348283044937798
EPOCH 22 MSE 0.4368944434872788
EPOCH 23 MSE 0.4238970813743784
EPOCH 24 MSE 0.4295895665066462
EPOCH 25 MSE 0.4208496700114882
EPOCH 26 MSE 0.4076942576872516
EPOCH 27 MSE 0.4016049619684728
EPOCH 28 MSE 0.4068153330871481
EPOCH 29 MSE 0.40480419046840677
EPOCH 30 MSE 0.39354828653327323
EPOCH 31 M

EPOCH 257 MSE 0.18404525245157374
EPOCH 258 MSE 0.18645889835420149
EPOCH 259 MSE 0.18466429605838539
EPOCH 260 MSE 0.18875819311324546
EPOCH 261 MSE 0.18430901495098478
EPOCH 262 MSE 0.18250927633034755
EPOCH 263 MSE 0.1865313529040061
EPOCH 264 MSE 0.18199546308112705
EPOCH 265 MSE 0.17829143723873808
EPOCH 266 MSE 0.18206092361783727
EPOCH 267 MSE 0.17785256594725876
EPOCH 268 MSE 0.18184727109293633
EPOCH 269 MSE 0.18377398661924618
EPOCH 270 MSE 0.1789912536448296
EPOCH 271 MSE 0.17528376570537324
EPOCH 272 MSE 0.176451704706052
EPOCH 273 MSE 0.1774652497606415
EPOCH 274 MSE 0.17939173652153326
EPOCH 275 MSE 0.18020269396797473
EPOCH 276 MSE 0.18420778949614633
EPOCH 277 MSE 0.17982029236311367
EPOCH 278 MSE 0.17967052415970314
EPOCH 279 MSE 0.17784461624147047
EPOCH 280 MSE 0.1821079822807092
EPOCH 281 MSE 0.17807014502037244
EPOCH 282 MSE 0.17783779811861766
EPOCH 283 MSE 0.17527333412230842
EPOCH 284 MSE 0.17701172969915888
EPOCH 285 MSE 0.18088370699161535
EPOCH 286 MSE 0.1855

EPOCH 507 MSE 0.15787456445950537
EPOCH 508 MSE 0.1605890744087361
EPOCH 509 MSE 0.15836794157126283
EPOCH 510 MSE 0.1588320590662242
EPOCH 511 MSE 0.1553346894817102
EPOCH 512 MSE 0.1556220315376701
EPOCH 513 MSE 0.15802403105672247
EPOCH 514 MSE 0.15813281132676804
EPOCH 515 MSE 0.15472101464209978
EPOCH 516 MSE 0.15305512629893347
EPOCH 517 MSE 0.15363104233902974
EPOCH 518 MSE 0.1501515609264662
EPOCH 519 MSE 0.1474348331870896
EPOCH 520 MSE 0.15164384301185496
EPOCH 521 MSE 0.1550504150753736
EPOCH 522 MSE 0.15825016448470267
EPOCH 523 MSE 0.15796733603827948
EPOCH 524 MSE 0.1617421866717244
EPOCH 525 MSE 0.1604362278558678
EPOCH 526 MSE 0.16379294085519613
EPOCH 527 MSE 0.16059725629062516
EPOCH 528 MSE 0.16100712029919811
EPOCH 529 MSE 0.15695451185653336
EPOCH 530 MSE 0.1547681417686137
EPOCH 531 MSE 0.14993930565294905
EPOCH 532 MSE 0.14793582332619507
EPOCH 533 MSE 0.15047572864246236
EPOCH 534 MSE 0.15069478582327173
EPOCH 535 MSE 0.14847266743749746
EPOCH 536 MSE 0.14938493

EPOCH 771 MSE 0.14044849218683217
EPOCH 772 MSE 0.13978028791494124
EPOCH 773 MSE 0.13825355794038902
EPOCH 774 MSE 0.13457936365653858
EPOCH 775 MSE 0.13502881855245938
EPOCH 776 MSE 0.13512071398880818
EPOCH 777 MSE 0.13265833248792847
EPOCH 778 MSE 0.1288858034717394
EPOCH 779 MSE 0.12785338762999773
EPOCH 780 MSE 0.12773670819174196
EPOCH 781 MSE 0.13049805029093992
EPOCH 782 MSE 0.1315022572760106
EPOCH 783 MSE 0.13592428642155654
EPOCH 784 MSE 0.13663870617518173
EPOCH 785 MSE 0.13824681366304345
EPOCH 786 MSE 0.13639016302879836
EPOCH 787 MSE 0.13681201912974325
EPOCH 788 MSE 0.1381240385654343
EPOCH 789 MSE 0.14077625019572704
EPOCH 790 MSE 0.14213337254965855
EPOCH 791 MSE 0.14145758275451614
EPOCH 792 MSE 0.13856366331981473
EPOCH 793 MSE 0.14156245312727944
EPOCH 794 MSE 0.14517378643064158
EPOCH 795 MSE 0.14291674308668398
EPOCH 796 MSE 0.14214643055553072
EPOCH 797 MSE 0.14160952266132576
EPOCH 798 MSE 0.14402071819439802
EPOCH 799 MSE 0.14383704809739173
EPOCH 800 MSE 0.1

EPOCH 1029 MSE 0.14144181997509542
EPOCH 1030 MSE 0.13792375274294738
EPOCH 1031 MSE 0.13703116876013752
EPOCH 1032 MSE 0.13947558670961877
EPOCH 1033 MSE 0.13790509283748995
EPOCH 1034 MSE 0.1383762737590311
EPOCH 1035 MSE 0.13960959011611476
EPOCH 1036 MSE 0.14194045599564128
EPOCH 1037 MSE 0.14223593834914464
EPOCH 1038 MSE 0.13899490400233575
EPOCH 1039 MSE 0.14052942403383578
EPOCH 1040 MSE 0.1402790476279428
EPOCH 1041 MSE 0.13880459515279547
EPOCH 1042 MSE 0.14211406334267948
EPOCH 1043 MSE 0.14436339578510712
EPOCH 1044 MSE 0.1406731358519812
EPOCH 1045 MSE 0.14138061286655143
EPOCH 1046 MSE 0.1417806369752487
EPOCH 1047 MSE 0.14325592149900634
EPOCH 1048 MSE 0.14159936652439897
EPOCH 1049 MSE 0.14121695176225096
EPOCH 1050 MSE 0.13966116709209087
EPOCH 1051 MSE 0.14349289975104318
EPOCH 1052 MSE 0.1475210598774569
EPOCH 1053 MSE 0.14356506931917373
EPOCH 1054 MSE 0.14700549693779114
EPOCH 1055 MSE 0.14510458123493272
EPOCH 1056 MSE 0.1462712495840806
EPOCH 1057 MSE 0.142785560

...

EPOCH 6446 MSE 0.16227330254590977
EPOCH 6447 MSE 0.164654935670251
EPOCH 6448 MSE 0.16087625573512995
EPOCH 6449 MSE 0.16675077362713833
EPOCH 6450 MSE 0.16258435869657553
EPOCH 6451 MSE 0.16788269285881016
EPOCH 6452 MSE 0.16556231474187558
EPOCH 6453 MSE 0.16430266559136802
EPOCH 6454 MSE 0.16577518118968126
EPOCH 6455 MSE 0.1694045022988885
EPOCH 6456 MSE 0.1732106791337972
EPOCH 6457 MSE 0.17298439314064762
EPOCH 6458 MSE 0.17160382617685677
EPOCH 6459 MSE 0.16907436447211396
EPOCH 6460 MSE 0.17250518231968012
EPOCH 6461 MSE 0.17252874398781548
EPOCH 6462 MSE 0.17627393597309662
EPOCH 6463 MSE 0.17218774627699895
EPOCH 6464 MSE 0.17088524025059526
EPOCH 6465 MSE 0.1751959780196959
EPOCH 6466 MSE 0.17632293211078726
EPOCH 6467 MSE 0.17210394454516603
EPOCH 6468 MSE 0.167408971849755
EPOCH 6469 MSE 0.16901201786176343
EPOCH 6470 MSE 0.17028799423955665
EPOCH 6471 MSE 0.16680887170801814
EPOCH 6472 MSE 0.17083531049543926
EPOCH 6473 MSE 0.1692601449947423
EPOCH 6474 MSE 0.17066477088

EPOCH 6695 MSE 0.18416025337572156
EPOCH 6696 MSE 0.1807068658221336
EPOCH 6697 MSE 0.17750760185443532
EPOCH 6698 MSE 0.17902636244026823
EPOCH 6699 MSE 0.18248375877707126
EPOCH 6700 MSE 0.18185708307333331
EPOCH 6701 MSE 0.17934314210525715
EPOCH 6702 MSE 0.1819116646532431
EPOCH 6703 MSE 0.18119593383989122
EPOCH 6704 MSE 0.18536338652500792
EPOCH 6705 MSE 0.18925034110422226
EPOCH 6706 MSE 0.1921808991264801
EPOCH 6707 MSE 0.18704348248104644
EPOCH 6708 MSE 0.18993089782275682
EPOCH 6709 MSE 0.18697312303811203
EPOCH 6710 MSE 0.1902808529127043
EPOCH 6711 MSE 0.19068810408256548
EPOCH 6712 MSE 0.18454388230367758
EPOCH 6713 MSE 0.1875366825328004
EPOCH 6714 MSE 0.18283189578356068
EPOCH 6715 MSE 0.18007299032161458
EPOCH 6716 MSE 0.17492996508466926
EPOCH 6717 MSE 0.1776681480793534
EPOCH 6718 MSE 0.1814248331509516
EPOCH 6719 MSE 0.18181655087740503
EPOCH 6720 MSE 0.18241021869760948
EPOCH 6721 MSE 0.18372767916958313
EPOCH 6722 MSE 0.18612357558800077
EPOCH 6723 MSE 0.1785850094

EPOCH 6949 MSE 0.177660833743864
EPOCH 6950 MSE 0.18117126716418017
EPOCH 6951 MSE 0.17792922216885199
EPOCH 6952 MSE 0.1755500978504145
EPOCH 6953 MSE 0.17835908632002004
EPOCH 6954 MSE 0.17412315122691463
EPOCH 6955 MSE 0.16715012025955708
EPOCH 6956 MSE 0.16378822194237974
EPOCH 6957 MSE 0.16943011064288593
EPOCH 6958 MSE 0.1720887073605663
EPOCH 6959 MSE 0.17446681620644283
EPOCH 6960 MSE 0.1708127523306161
EPOCH 6961 MSE 0.16393634737529716
EPOCH 6962 MSE 0.15733920291224548
EPOCH 6963 MSE 0.1598729281577892
EPOCH 6964 MSE 0.15818695130699906
EPOCH 6965 MSE 0.160907500105608
EPOCH 6966 MSE 0.16627509840420054
EPOCH 6967 MSE 0.17137880103843847
EPOCH 6968 MSE 0.17221298674861227
EPOCH 6969 MSE 0.17555164344516797
EPOCH 6970 MSE 0.17155852286501136
EPOCH 6971 MSE 0.17485966804692796
EPOCH 6972 MSE 0.17361954037024366
EPOCH 6973 MSE 0.17758531451527432
EPOCH 6974 MSE 0.17390625756216715
EPOCH 6975 MSE 0.17765124527926648
EPOCH 6976 MSE 0.18036575267163696
EPOCH 6977 MSE 0.18397854721

EPOCH 7187 MSE 0.17476867920317724
EPOCH 7188 MSE 0.17370709430830508
EPOCH 7189 MSE 0.1775459271094586
EPOCH 7190 MSE 0.17212912448094886
EPOCH 7191 MSE 0.1669610702452035
EPOCH 7192 MSE 0.16203010975638182
EPOCH 7193 MSE 0.1587283580863458
EPOCH 7194 MSE 0.1618464475578866
EPOCH 7195 MSE 0.16594027676453627
EPOCH 7196 MSE 0.1630172363648259
EPOCH 7197 MSE 0.16592540841794864
EPOCH 7198 MSE 0.16171921852160975
EPOCH 7199 MSE 0.16249825897803336
EPOCH 7200 MSE 0.16529525705654088
EPOCH 7201 MSE 0.16713706145067303
EPOCH 7202 MSE 0.16903603773197426
EPOCH 7203 MSE 0.17166297220930804
EPOCH 7204 MSE 0.17412531085231286
EPOCH 7205 MSE 0.17301203726682432
EPOCH 7206 MSE 0.16595827776951658
EPOCH 7207 MSE 0.1710136232037273
EPOCH 7208 MSE 0.16679646251532132
EPOCH 7209 MSE 0.1686279591225546
EPOCH 7210 MSE 0.16789504440917044
EPOCH 7211 MSE 0.1730966326313837
EPOCH 7212 MSE 0.17640280492628424
EPOCH 7213 MSE 0.17145765767196855
EPOCH 7214 MSE 0.16765268043253778
EPOCH 7215 MSE 0.17357277018

EPOCH 7434 MSE 0.17305594686133077
EPOCH 7435 MSE 0.1746895392588063
EPOCH 7436 MSE 0.17855014364314306
EPOCH 7437 MSE 0.1821847607420647
EPOCH 7438 MSE 0.17985820308699163
EPOCH 7439 MSE 0.17742911270910888
EPOCH 7440 MSE 0.18060499763248156
EPOCH 7441 MSE 0.17890036650235835
EPOCH 7442 MSE 0.17473477090869605
EPOCH 7443 MSE 0.1793957712710993
EPOCH 7444 MSE 0.17554255969266888
EPOCH 7445 MSE 0.17809742453548275
EPOCH 7446 MSE 0.17832575466811496
EPOCH 7447 MSE 0.1753820886740399
EPOCH 7448 MSE 0.17315960829442237
EPOCH 7449 MSE 0.17798989177453464
EPOCH 7450 MSE 0.17662551333718057
EPOCH 7451 MSE 0.17825980475591163
EPOCH 7452 MSE 0.17443784553658961
EPOCH 7453 MSE 0.17008947827124823
EPOCH 7454 MSE 0.16769884843474947
EPOCH 7455 MSE 0.17009064283821873
EPOCH 7456 MSE 0.16886460779313145
EPOCH 7457 MSE 0.16790803374326838
EPOCH 7458 MSE 0.17343781124884955
EPOCH 7459 MSE 0.1700734440060806
EPOCH 7460 MSE 0.1703052949648507
EPOCH 7461 MSE 0.1719028435191603
EPOCH 7462 MSE 0.1760421110